Use Case 9
==========

Problem Definition:
-------------------

    A climate scientist wishes to analyse potential correlations between Aerosol and Cloud ECVs.
    
Required Toolbox Features:
--------------------------

    * Access to and ingestion of ESA CCI Aerosol and Cloud data (Aerosol Optical Depth and Cloud Fraction)
    * Geometric adjustments
    * Spatial (point, polygon) and temporal subsetting
    * Visualisation of both times series at the same time: e.g. time series plot, time series animation
    * Correlation analysis, scatter-plot of correlation statistics, saving of image and correlation statistics on disk (format options)

Ingest data and create datasets
-------------------------------

In [ ]:
from ect.core.io import DATA_STORE_REGISTRY
from ect.core.monitor import ConsoleMonitor
import ect.ops

data_store = DATA_STORE_REGISTRY.get_data_store('default')
data_store

In [ ]:
data_source_aerosol = data_store.query('AEROSOL_AATSR_SU_L3_V4.21_MONTHLY')
data_source_clouds = data_store.query('CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0')

Retrieve data by synchronizing with the ftp server

In [ ]:
monitor = ConsoleMonitor()
#data_source_aerosol[0].sync(('2007-01-01','2009-01-01'), monitor)

In [ ]:
#data_source_clouds[0].sync(('2007-01-01','2009-01-01'), monitor)

In [ ]:
time_range = tuple(['2008-01-01','2009-01-01'])
dataset_aerosol = data_source_aerosol[0].open_dataset(time_range)
dataset_clouds = data_source_clouds[0].open_dataset(time_range)

In [ ]:
print(dataset_clouds.dims)
print(dataset_aerosol.dims)

In [ ]:
print(dataset_clouds.data_vars)

Filter datasets to select the desired variables
-----------------------------------------------

In [ ]:
clouds_filter = tuple(['cc_total'])
aerosol_filter = tuple(['AOD550_mean'])

dataset_clouds = ect.ops.filter_dataset(dataset_clouds, clouds_filter)
dataset_aerosol = ect.ops.filter_dataset(dataset_aerosol, aerosol_filter)

In [ ]:
print(dataset_clouds.dims)
print(dataset_aerosol.dims)

In [ ]:
ect.ops.harmonize([dataset_clouds, dataset_aerosol])

In [ ]:
print(dataset_clouds.dims)
print(dataset_aerosol.dims)

Plot the first time slice of the dataset

In [ ]:
%matplotlib inline
ect.ops.plot_map(dataset_clouds, variable='cc_total', time=0, path='/home/ccitbx/Desktop/fig1.png')

In [ ]:
ect.ops.plot_map(dataset_aerosol, variable='AOD550_mean', time=0, path='/home/ccitbx/Desktop/fig2.png')

Co-register datasets by resampling
----------------------------------

In [ ]:
dataset_aerosol = ect.ops.coregister(dataset_clouds, dataset_aerosol, 'bilinear')

In [ ]:
print(dataset_clouds.dims)
print(dataset_aerosol.dims)

In [ ]:
ect.ops.plot_map(dataset_aerosol, variable='AOD550_mean', time=0, path='/home/ccitbx/Desktop/fig3.png')

Select the desired spatial region
---------------------------------

In [ ]:
dataset_aerosol = ect.ops.subset_spatial(dataset_aerosol, lat=[-40.,40.], lon=[-20.,60.])
dataset_clouds = ect.ops.subset_spatial(dataset_clouds, lat=[-40.,40.], lon=[-20.,60.])

In [ ]:
print(dataset_clouds.dims)
print(dataset_aerosol.dims)

In [ ]:
ect.ops.plot_map(dataset_clouds, variable='cc_total', time=0, path='/home/ccitbx/Desktop/fig4.png')

In [ ]:
ect.ops.plot_map(dataset_clouds, variable='cc_total', time=0, 
                 extents=[-20., 60., -40., 40.], path='/home/ccitbx/Desktop/fig5.png')

Select the desired temporal region
----------------------------------

In [ ]:
dataset_aerosol = ect.ops.subset_temporal_index(dataset_aerosol, time=[0,10])
dataset_clouds = ect.ops.subset_temporal_index(dataset_clouds, time=[0,10])

In [ ]:
print(dataset_clouds.dims)
print(dataset_aerosol.dims)

In [ ]:
print(dataset_clouds.time)
print(dataset_aerosol.time)

Retrieve and plot timeseries
----------------------------

In [ ]:
timeseries_aerosol_point = ect.ops.timeseries(dataset_aerosol, lat=50., lon=50.)
timeseries_clouds_point = ect.ops.timeseries(dataset_clouds, lat=50., lon=50.)
timeseries_aerosol_mean = ect.ops.timeseries_mean(dataset_aerosol)
timeseries_clouds_mean = ect.ops.timeseries_mean(dataset_clouds)

In [ ]:
print(timeseries_aerosol_mean)
print(timeseries_clouds_mean)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(timeseries_aerosol_mean['AOD550_mean'].data)

In [ ]:
plt.plot(timeseries_clouds_mean['cc_total'].data)

Do correlation analysis
-----------------------

In [ ]:
correlation = ect.ops.pearson_correlation(timeseries_clouds_mean, timeseries_aerosol_mean, 
                                          path="/home/ccitbx/Desktop/a.txt")
correlation

Create a scatter plot
---------------------------------